In [3]:
import pandas as pd
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [9]:
# json_file = Path()
df = pd.DataFrame(columns=['date', 'chamber', 'speaker', 'speaker_id','text'])
json_file =  Path.cwd().parent.joinpath(r'output\2020\CREC-2020-01-08\json\CREC-2020-01-08-pt1-PgE11.json')
with open(json_file) as f:
    data = json.load(f)
    spech_date = data['header']['month'] + ' ' + data['header']['day'] + ', ' + data['header']['year']
    chamber = data['header']['chamber']
    for speech in data['content']:
        if speech['kind'] == 'speech':
            speaker = speech['speaker']
            speaker_id = speech['speaker_bioguide']
            text = speech['text']
            df = df.append({'date': spech_date, 'chamber': chamber, 
                            'speaker': speaker, 'speaker_id': speaker_id, 
                            'text': text}, ignore_index=True)
df

C:\Users\jesse\AppData\Local\Temp\ipykernel_30444\3475167099.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': spech_date, 'chamber': chamber,


,date,chamber,speaker,speaker_id,text
0,"January 8, 2020",House,Mr. GONZALEZ of Texas,G000581,"Mr. GONZALEZ of Texas. Madam Speaker, I was ..."


In [2]:
def scrape_speeches(json_file):
    """
    Scrape speech text from an individual json file
    """
    temp_df = pd.DataFrame(columns=['date', 'chamber', 'speaker', 'speaker_id','text'])
    with open(json_file) as f:
        data = json.load(f)
        spech_date = data['header']['month'] + ' ' + data['header']['day'] + ', ' + data['header']['year']
        chamber = data['header']['chamber']
        for speech in data['content']:
            if speech['kind'] == 'speech':
                speaker = speech['speaker']
                speaker_id = speech['speaker_bioguide']
                text = speech['text']
                temp_df = temp_df.append({'date': spech_date, 'chamber': chamber, 
                                'speaker': speaker, 'speaker_id': speaker_id, 
                                'text': text}, ignore_index=True)
    return temp_df

In [3]:
# iterate through all json files in the output folder
# and scrape the speech text
speech_df = pd.DataFrame(columns=['date', 'chamber', 'speaker', 'speaker_id','text'])
for json_file in Path.cwd().parent.joinpath(r'output').rglob('*.json'):
    temp_df = scrape_speeches(json_file)
    speech_df = speech_df.append(temp_df, ignore_index=True)
speech_df

,date,chamber,speaker,speaker_id,text
0,"January 2, 2015",House,Mr. BOEHNER,B000589,"Mr. BOEHNER. Mr. Speaker, I rise today to ho..."
1,"January 2, 2015",House,Mr. KINGSTON,None,"Mr. KINGSTON. Mr. Speaker, I rise today in s..."
2,"January 2, 2015",House,Mr. WAXMAN,W000215,"Mr. WAXMAN. Mr. Speaker, since before the te..."
3,"January 2, 2015",House,Mr. VAN HOLLEN,V000128,"Mr. VAN HOLLEN. Mr. Speaker, I rise today to..."
4,"January 2, 2015",House,Mr. CONNOLLY,C001078,"Mr. CONNOLLY. Mr. Speaker, I ask my colleagu..."
...,...,...,...,...,...
354183,"February 16, 2023",Senate,The PRESIDING OFFICER (Mr. Cardin),None,The PRESIDING OFFICER (Mr. Cardin). The Sena...
354184,"February 16, 2023",Senate,Mr. MURPHY,M001169,"Mr. MURPHY. Mr. President, I rise today to o..."
354185,"February 16, 2023",Senate,Mr. CARDIN,C000141,"Mr. CARDIN. Mr. President, I ask unanimous c..."
354186,"February 16, 2023",Senate,The PRESIDING OFFICER,None,"The PRESIDING OFFICER. Without objection, it..."


In [4]:
# filter out rows where the word 'None' is contained in the speaker_id
speech_df = speech_df[~speech_df['speaker_id'].fillna('None').str.contains('None')]
speech_df.reset_index(drop=True, inplace=True)
speech_df['text_len'] = speech_df['text'].str.len()
speech_df.sort_values(by='text_len', ascending=False, inplace=True)
speech_df

,date,chamber,speaker,speaker_id,text,text_len
96049,"April 4, 2017",Senate,Mr. MERKLEY,M001176,Mr. MERKLEY. Let's turn to Planned Parenthoo...,291680
96048,"April 4, 2017",Senate,Mr. MERKLEY,M001176,"Mr. MERKLEY. Mr. President, I rise to addres...",224212
131713,"February 7, 2018",House,Ms. PELOSI,P000197,"Ms. PELOSI. Mr. Speaker, I thank the gentlew...",194534
19723,"May 20, 2015",Senate,Mr. PAUL,P000603,"Mr. PAUL. Mr. President, there comes a time ...",125903
131718,"February 7, 2018",House,Ms. PELOSI,P000197,Ms. PELOSI. Aldo Solano was at the State of ...,120007
...,...,...,...,...,...,...
110267,"July 27, 2017",Senate,Mr. ENZI,E000285,Mr. ENZI. No.,15
110250,"July 27, 2017",Senate,Mr. ENZI,E000285,Mr. ENZI. No.,15
233677,"October 21, 2021",House,Mr. ROY,R000614,Mr. ROY. Yes.,15
233669,"October 21, 2021",House,Mr. ROY,R000614,Mr. ROY. No.,14


In [5]:
# for year in speech_df['date'], split df by year and save comprressed csv files
for year in speech_df['date'].str.split().str[2].unique():
    temp_df = speech_df[speech_df['date'].str.contains(year)]
    temp_df.reset_index(drop=True, inplace=True)
    temp_df.to_csv(f'congressional_record_speeches_{year}.csv.gzip', compression='gzip', index=False)